# OpenML CC18 Metalearning Benchmark

In [1]:
%matplotlib inline

In [28]:
import pandas as pd
import pandera as pa
import plotly.express as px

## Load Data

In [215]:
JOB = 312


check_betw_zero_one = pa.Check(lambda s: (0 <= s) & (s <= 1))

INFERENCE_RESULT_SCHEMA = pa.DataFrameSchema(
    columns={
        "data_env": pa.Column(pa.String),
        "n_inference_steps": pa.Column(pa.Int),
        "is_valid": pa.Column(pa.Bool),
        "reward": pa.Column(pa.Float, check_betw_zero_one),
        "validation_score": pa.Column(pa.Float, check_betw_zero_one),
    },
    coerce=True,
)

@pa.check_output(INFERENCE_RESULT_SCHEMA)
def data_to_longform(inference_results):
    return (
        inference_results
        [["data_env", "n_inference_steps", "key", "value"]]
        .dropna(subset=["value"])
        .pivot_table(
            index=["data_env", "n_inference_steps"],
            columns="key",
            values="value",
            aggfunc=lambda x: x,
        )
        .reset_index()
        .dropna()
        # all scores should be strings
        .loc[
            lambda df: (
                df.validation_score.map(lambda x: isinstance(x, str)) &
                df.mlf_description.map(lambda x: isinstance(x, str)) &
                df.reward.map(lambda x: isinstance(x, str)) &
                df.is_valid.map(lambda x: isinstance(x, str))
            )
        ]
        .rename_axis(None, axis=1)
    )

In [216]:
inference_results = pd.concat([
    (
        pd.read_csv(f"../floyd_outputs/{JOB}/{env}_env_inference_results.csv")
        .pipe(data_to_longform)
        .assign(data_env_partition=env)
    ) for env in ["training", "test"]
])
inference_results.head()

,data_env,n_inference_steps,is_valid,mlf_description,reward,validation_score,data_env_partition
200,kaggle.santander_customer_satisfaction,0,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.766179,0.766179,training
201,kaggle.santander_customer_satisfaction,1,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.800464,0.800464,training
202,kaggle.santander_customer_satisfaction,2,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.792258,0.792258,training
203,kaggle.santander_customer_satisfaction,3,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.794338,0.794338,training
204,kaggle.santander_customer_satisfaction,4,True,{'SimpleImputer': ColumnTransformer(n_jobs=Non...,0.742671,0.742671,training


In [189]:
inference_results.dtypes

data_env               object
n_inference_steps       int64
is_valid                 bool
mlf_description        object
reward                float64
validation_score      float64
data_env_partition     object
dtype: object

## Plot Rewards

In [214]:
inference_results.query("data_env_partition == 'training'").data_env.str.split(".").str[0]

200     kaggle
201     kaggle
202     kaggle
203     kaggle
204     kaggle
         ...  
8595    openml
8596    openml
8597    openml
8598    openml
8599    openml
Name: data_env, Length: 6697, dtype: object

In [236]:
px.line(
    (
        inference_results
        .query("data_env_partition == 'training'")
        .query("data_env.str.startswith('kaggle.')")
    ),
    x="n_inference_steps",
    y="validation_score",
    template="plotly_white",
    facet_col="data_env",
    facet_col_wrap=2,
)

### Plot of Test Data Environment Validation Scores

In [235]:
px.line(
    inference_results.query("data_env_partition == 'test'"),
    x="n_inference_steps",
    y="validation_score",
    template="plotly_white",
    facet_col="data_env",
    facet_col_wrap=3,
    height=1500,
)